In [1]:
import os
import re
import glob
import feather
import numpy as np
import pandas as pd
import unicodedata
import matplotlib.pyplot as plt
from itertools import product
from functools import partial
from pathlib import Path

# dask
import dask
import dask.dataframe as dd
from dask.distributed import Client


from src.clean_data import clean_rfc_fantasma, clean_sancionados, clean_rupc

# features generales
from src.features_general import (
    monto_por_unidad_compradora,
    proveedores_distintos,
    procedimientos_distintos,
    numero_de_contratos,
)

# features competencia
from src.features_competencia import (
    contratos_por_proveedor,
    porcentaje_procedimientos_por_tipo,
    porcentaje_monto_tipo_procedimiento,
    importe_promedio_por_contrato,
    calcular_IHH_ID_contratos,
    calcular_IHH_ID_monto,
    tendencia_adjudicacion_directa,
    # participantes
    porcentaje_licitaciones_con_un_participante,
    procedimientos_promedio_por_participantes,
    indice_participacion,
    procedimientos_por_participantes_unicos,
    tendencia_incremento_participantes,
)

# features transparencia
from src.features_transparencia import (
    # procedimientos
    porcentaje_procedimientos_presenciales,
    contratos_promedio_por_procedimimento,
    # contratos_por_duracion,
    # monto_por_duracion,
    promedio_datos_faltantes_poc_contrato,
    # scraper
    porcentaje_procs_sin_contrato,
    porcentaje_procs_sin_fallo,
    porcentaje_procs_sin_apertura,
    porcentaje_procedimientos_sin_archivos,
    promedio_procs_por_archivo,
    tendencia_no_publicacion_contratos,
    porcentaje_adjudicaciones_incompletas,
    porcentaje_invitaciones_incompletas,
    porcentaje_licitaciones_incompletas,
)

# features anomalias
from src.features_anomalias import (
    # procedimientos
    interaccion_rfc_fantasma,
    interaccion_sancionados,
    porcentaje_contratos_por_convenio,
    pc_licitaciones_nacionales_menor_15_dias,
    pc_licitaciones_internacionales_menor_20_dias,
    pc_licitaciones_internacionales_menor_40_dias,
    diferente_estratificacion,
    porcentaje_adjudicaciones_excedieron_monto,
    porcentaje_invitaciones_excedieron_monto,
    # scraper
    promedio_convenios_por_proc,
    porcentaje_procs_sin_junta_aclaracion,
    porcentaje_procs_sin_convocatoria,
)


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline
%config IPCompleter.use_jedi = False
%load_ext autoreload

%autoreload 2

client = Client()
client

Client Scheduler: tcp://127.0.0.1:37532 Dashboard: http://127.0.0.1:8787,Cluster Workers: 8 Cores: 8 Memory: 16.69 GB


### Procedimientos

In [2]:
df_procedimientos = feather.read_dataframe(
    '../data/bases/procedimientos_all_2017_10_29.feather', nthreads=7)
print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.loc[df_procedimientos.GOBIERNO == 'APF', :]
df_procedimientos = df_procedimientos.drop('GOBIERNO', axis=1)
df_procedimientos = df_procedimientos.assign(CLAVEUC=df_procedimientos.CLAVEUC_REAL)
df_procedimientos = df_procedimientos.drop('CLAVEUC_REAL', axis=1)
df_procedimientos = df_procedimientos.loc[~df_procedimientos.PROVEEDOR_CONTRATISTA.isnull()]
print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.loc[df_procedimientos.CLAVEUC != 'MISSING']

# Rellenar el caracter faltante
caracter = df_procedimientos.CARACTER.mask(df_procedimientos.CARACTER == '').fillna('SIN REPORTAR')
df_procedimientos = df_procedimientos.assign(CARACTER=caracter)

mapa_codigos_caracter = {'N': 'NACIONAL', 'I': 'INTERNACIONAL', 'T': 'INTERNACIONAL BAJO TLC'}

codigos_caracter = df_procedimientos.loc[df_procedimientos.CARACTER == 'SIN REPORTAR'].NUMERO_PROCEDIMIENTO.map(
    lambda x: x.split('-')[2][0] if isinstance(x, str) else x
)
codigos_caracter = codigos_caracter.map(lambda x: mapa_codigos_caracter.get(x, 'SIN REPORTAR'))

df_procedimientos.loc[df_procedimientos.CARACTER == 'SIN REPORTAR', 'CARACTER'] = codigos_caracter

del caracter, mapa_codigos_caracter, codigos_caracter
print(df_procedimientos.shape)
df_procedimientos.head()

(1014554, 47)
(884997, 45)
(884960, 45)


,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,IMPORTE_PESOS
70,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,nan,MULTISERVICIOS Y ASESORIAS DEL CARIBE,HABILITADO,None,319200.0,NaN,MXN,IA-009JZL032-N1-2012,MIXTA,INVITACION A CUANDO MENOS TRES,165782,SERVICIO DE LIMPIEZA CZM 2012,None,ADQUISICIONES,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,NaT,2011-12-16 00:00:00,2011-12-19,NaT,2012-01-01,2013-02-28,115807,SERVICIO DE LIMPIEZA CZM 2012,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,319200.0
71,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,nan,ALEJANDRO ALARCON JIMON,HABILITADO,None,26097.0,NaN,MXN,AA-009JZL032-N4-2012,PRESENCIAL,ADJUDICACION DIRECTA,165738,MANTENIMIENTO A EXTINTORES CZM,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO,2012-08-10 15:59:00,2012-08-10 16:05:00,NaT,2012-08-20,2012-08-26,2012-12-31,242370,SERVICIO DE MANTENIMIENTO A EXTINTORES AD CZM ...,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,26097.0
72,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,5578,TRANSPORTE ESPECIALIZADO DE PERSONAL NACIONAL,HABILITADO,MEDIANA,361200.0,NaN,MXN,IA-009JZL032-N2-2012,MIXTA,INVITACION A CUANDO MENOS TRES,148131,SERVICIO DE TRANSPORTE DE PERSONAL DE LA ESTAC...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,2011-12-09 14:51:00,2011-12-16 14:00:00,2011-12-19,NaT,2012-01-01,2013-02-28,116667,SERVICIO DE TRANSPORTE DE PERSONAL CZM 2012,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,361200.0
73,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,nan,COMERCIALIZADORA EL MAHARAJA,HABILITADO,None,334264.0,NaN,MXN,AA-009JZL032-N5-2011,MIXTA,ADJUDICACION DIRECTA,276283,SERVICIO DE COMEDOR DEL PERSONAL DE LA ESTACIO...,None,ADQUISICIONES,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,2011-12-28 12:45:00,2011-12-28 17:00:00,NaT,NaT,2012-01-01,2013-02-28,123933,SERVICIO DE COMEDOR AD CZM 2012,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,334264.0
74,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MEDIANA,nan,COMERCIALIZADORA EL MAHARAJA,HABILITADO,None,285948.0,NaN,MXN,AA-009JZL032-N5-2011,MIXTA,ADJUDICACION DIRECTA,148083,SERVICIO DE COMEDOR PARA EL PERSONAL DE LA EST...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,NaN,NO,2011-12-28 12:45:00,2011-12-28 17:00:00,NaT,2011-12-30,2012-01-01,2012-12-31,123933,SERVICIO DE COMEDOR AD CZM 2012,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,285948.0


### Scraper

In [3]:
cols_scraper = [
    'CODIGO_EXPEDIENTE',
    'archivo_anexos', 'archivo_apertura',
    'archivo_contrato', 'archivo_convocatoria',
    'archivo_fallo', 'archivo_junta',
    'numero_archivos','numero_convenios',
]

df_scraper = pd.read_csv(
    '../data/bases/tabla_scraper_2017_10_29.csv',
    usecols=cols_scraper, dtype={'CODIGO_EXPEDIENTE': str, 'Year': str}
)
print(df_scraper.shape)
df_scraper = df_scraper.loc[:, cols_scraper]

## Join con procedimientos
cols_procs=[
    'CLAVEUC', 'NUMERO_PROCEDIMIENTO', 'CODIGO_EXPEDIENTE',
    'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION'
]

df_contratos = (df_procedimientos.groupby(['CLAVEUC', 'NUMERO_PROCEDIMIENTO',
                                           'CODIGO_EXPEDIENTE', 'TIPO_PROCEDIMIENTO',
                                           'TIPO_CONTRATACION'], as_index=False).CODIGO_CONTRATO.count()
                                 .rename(columns={'CODIGO_CONTRATO': 'numero_contratos'}))
print(df_contratos.shape)
df_fechas = df_procedimientos[['CODIGO_EXPEDIENTE', 'FECHA_INICIO']].drop_duplicates(subset=['CODIGO_EXPEDIENTE'])
df_contratos = pd.merge(df_contratos, df_fechas, on='CODIGO_EXPEDIENTE', how='inner')
print(df_contratos.shape)
df_scraper = pd.merge(df_scraper, df_contratos, on='CODIGO_EXPEDIENTE', how='inner')
del df_contratos, df_fechas
print(df_scraper.shape)
df_scraper.head(3)

(1102004, 9)
(695113, 6)
(695113, 7)
(633906, 15)


,CODIGO_EXPEDIENTE,archivo_anexos,archivo_apertura,archivo_contrato,archivo_convocatoria,archivo_fallo,archivo_junta,numero_archivos,numero_convenios,CLAVEUC,NUMERO_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,numero_contratos,FECHA_INICIO
0,599,0,1,0,0,1,1,58,0,018TOQ054,LO-018TOQ054-T2-2010,LICITACION PUBLICA,OBRA PUBLICA,1,2012-01-29
1,1171,0,1,1,1,1,1,6,1,041A00001,LA-010A00001-N3-2011,LICITACION PUBLICA,SERVICIOS,1,2012-01-01
2,1610,0,0,1,0,0,0,3,0,018TOQ999,IA-018TOQ999-I2-2011,INVITACION A CUANDO MENOS TRES,SERVICIOS,1,2014-03-15


### Participantes

In [4]:
cols_parts = [
    'NUMERO_PROCEDIMIENTO',
    'CODIGO_EXPEDIENTE', 
    'CLAVEUC',
    'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION', 'FORMA_PROCEDIMIENTO',
    'PROVEEDOR_CONTRATISTA',
    'ESTATUS_DE_PROPUESTA',
    'ESTATUS_PARTIDA',
    'ESTATUS_FALLO',
    'PRECIO_TOTAL',
]

df_participantes = dd.read_csv(
    '../data/processed/participantes/participantes_*_*.psv', sep='|', usecols=cols_parts,
    dtype={
        'TIPO_PROCEDIMIENTO': str,
        'TIPO_CONTRATACION': str,
        'FORMA_PROCEDIMIENTO': str,
        'ESTATUS_DE_PROPUESTA': str,
        'PERIODO': str,
        'DESCRIPCION_CUCOP': str,
        'PROVEEDOR_CONTRATISTA': str,
        'CODIGO_EXPEDIENTE': str
    }
)

df_participantes = df_participantes.loc[~df_participantes.PROVEEDOR_CONTRATISTA.isnull()]
df_participantes = df_participantes.loc[~df_participantes.CLAVEUC.isnull()]
df_participantes = df_participantes.loc[:, cols_parts]
df_participantes = df_participantes.assign(
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.fillna('SIN REPORTAR'),
    ESTATUS_DE_PROPUESTA=df_participantes.ESTATUS_DE_PROPUESTA.fillna('SIN REPORTAR'),
)
df_participantes = df_participantes.compute()

# Se saca el valor de TIPO_CONTRATACION del número de procedimiento
claves_procs = {
    'L': 'LICITACION PUBLICA',
    'I': 'INVITACION A CUANDO MENOS TRES',
    'A': 'ADJUDICACION DIRECTA',
    'O': 'LICITACION PUBLICA CON OSD',
    'S': 'ADJUDICACION DIRECTA',
    'P': 'PROYECTO DE CONVOCATORIA',
    'E': 'LICITACION PUBLICA',
    'X': 'SIN REPORTAR',
}
claves_contrs = {
    'A': 'ADQUISICIONES',
    'O': 'OBRA PUBLICA',
    'X': 'SIN REPORTAR'
}

claves = df_participantes.NUMERO_PROCEDIMIENTO.map(lambda x: x.split('-')[0])
procedimientos = claves.map(lambda x: claves_procs.get(x[0], None))
contrataciones = claves.map(lambda x: claves_contrs.get(x[1], None))

df_participantes = df_participantes.assign(
    TIPO_PROCEDIMIENTO=df_participantes.TIPO_PROCEDIMIENTO.mask(
        df_participantes.TIPO_PROCEDIMIENTO == 'SIN PLANTILLA', procedimientos),
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.mask(
        df_participantes.TIPO_CONTRATACION == 'SIN REPORTAR', contrataciones)
)

df_participantes = df_participantes.assign(
    TIPO_PROCEDIMIENTO=df_participantes.TIPO_PROCEDIMIENTO.fillna('SIN REPORTAR'),
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.fillna('SIN REPORTAR'),
)

del claves, procedimientos, contrataciones, cols_parts

# PROCEDIMIENTOS
col_proc = 'NUMERO_PROCEDIMIENTO'
 
df_procs_aux = feather.read_dataframe(
    '../data/bases/procedimientos_all_2017_10_29.feather', nthreads=7,
    columns=[
        col_proc,
        'TIPO_PROCEDIMIENTO',
        'TIPO_CONTRATACION'
    ]
)
print(df_procs_aux.shape)
df_procs_aux = df_procs_aux.drop_duplicates(subset=[col_proc])
print(df_procs_aux.shape)
df_procs_aux = df_procs_aux.rename(
    columns={'TIPO_CONTRATACION': 'TIPO_CONTRATACION_AUX',
             'TIPO_PROCEDIMIENTO': 'TIPO_PROCEDIMIENTO_AUX'}
)

df_participantes = pd.merge(df_participantes,
                            df_procs_aux,
                            on='NUMERO_PROCEDIMIENTO', how='left')

del df_procs_aux

df_participantes = df_participantes.assign(
    TIPO_PROCEDIMIENTO=df_participantes.TIPO_PROCEDIMIENTO.mask(
        df_participantes.TIPO_PROCEDIMIENTO == 'SIN REPORTAR',
        df_participantes.TIPO_PROCEDIMIENTO_AUX).fillna('SIN REPORTAR'),
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.mask(
        df_participantes.TIPO_CONTRATACION == 'SIN REPORTAR',
        df_participantes.TIPO_CONTRATACION_AUX).fillna('SIN REPORTAR'),
)

df_participantes = df_participantes.drop(['TIPO_PROCEDIMIENTO_AUX',
                                          'TIPO_CONTRATACION_AUX'], axis=1)

# Se cambia el valor al normalizado

df_participantes.loc[
    df_participantes.TIPO_PROCEDIMIENTO == 'INVITACION A CUANDO MENOS TRES', 'TIPO_PROCEDIMIENTO'
] = 'INVITACION A CUANDO MENOS TRES'

df_participantes.loc[
    df_participantes.TIPO_CONTRATACION == 'SERVICIO DE OBRA PUBLICA', 'TIPO_CONTRATACION'
] = 'SERVICIOS RELACIONADOS CON LA OP'

print(df_participantes.shape)
df_participantes.head()

(1014554, 3)
(804999, 3)
(5216946, 11)


,NUMERO_PROCEDIMIENTO,CODIGO_EXPEDIENTE,CLAVEUC,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,FORMA_PROCEDIMIENTO,PROVEEDOR_CONTRATISTA,ESTATUS_DE_PROPUESTA,ESTATUS_PARTIDA,ESTATUS_FALLO,PRECIO_TOTAL
0,IO-009000972-N30-2011,93548,009000972,INVITACION A CUANDO MENOS TRES,SERVICIOS RELACIONADOS CON LA OP,PRESENCIAL,CONSTRUCCIONES Y PAVIMENTOS BRUFA,GANADOR,NO ADJUDICADO,GANADOR,8963339.44
1,IO-009000972-N30-2011,93548,009000972,INVITACION A CUANDO MENOS TRES,SERVICIOS RELACIONADOS CON LA OP,PRESENCIAL,CONSORCIO CONSTRUCTOR LOBO,PERDEDOR,NO ADJUDICADO,GANADOR,10253719.14
2,IO-009000972-N30-2011,93548,009000972,INVITACION A CUANDO MENOS TRES,SERVICIOS RELACIONADOS CON LA OP,PRESENCIAL,CONSTRUCCIONES CESAR VALENZUELA VELASCO,PERDEDOR,NO ADJUDICADO,GANADOR,10104415.88
3,LA-006HHG001-N64-2011,97983,006HHG001,LICITACION PUBLICA,ADQUISICIONES,MIXTA,ASESORIA EN COMPUTACION SERVICIOS Y ELECTRONICA,GANADOR,NO ADJUDICADO,GANADOR,1970.00
4,AA-018T4I008-N8-2012,135592,018T4I008,ADJUDICACION DIRECTA,ADQUISICIONES,MIXTA,MEDICAL DEPOT,PERDEDOR,NO ADJUDICADO,GANADOR,9343.80


### RFC y Sancionados

In [5]:
# RFC fantasma
df_fantasma = pd.read_csv(
    '../data/bases/RFC fantasma.csv', parse_dates=['Publicación página SAT definitivos'],
    usecols=['RFC', 'Nombre del Contribuyente', 'Publicación página SAT definitivos'],
    dtype={'RFC': str, 'Nombre del Contribuyente': str}, encoding='iso-8859-1', skiprows=2
)
df_fantasma = df_fantasma.rename(columns={'Nombre del Contribuyente': 'Nombre del Contribuyente'.upper()})
df_fantasma = df_fantasma.loc[df_fantasma.RFC != 'XXXXXXXXXXXX']
df_fantasma = clean_rfc_fantasma(df_fantasma)

# Proveedores sancionados
df_sancionados = pd.read_excel('../data/bases/SancionProveedoresContratistas.xls')
df_sancionados = clean_sancionados(df_sancionados)
# Hay un registro con expediente vacío, de la tabla de procedimientos se observa
# que los contratos los realizó en 2014
df_sancionados.loc[
    df_sancionados.PROVEEDOR_CONTRATISTA == 'ALBA MARIA DE LA ASUNCION HERRASTI FERNANDEZ', 'Expediente'
] = 'XXXX/2014'
df_sancionados = df_sancionados.assign(
    Year=df_sancionados.Expediente.map(
        lambda x: int(x.split('/')[1])
    )
)

# poner el valor predeterminado para las funciones que usan estas tablas
interaccion_rfc_fantasma = partial(interaccion_rfc_fantasma, df_rfc_fantasma=df_fantasma)
interaccion_sancionados = partial(interaccion_sancionados, df_sancionados=df_sancionados)

### Montos máximos

In [6]:
df_montos_maximos = pd.read_csv('../data/bases/Montos_maximos.csv',
                                usecols=['Año', 'Tipo de contratación', ' Adjudicación directa ', ' INV3 '],
                                dtype={'Año': int},
                                encoding='iso-8859-1')
df_montos_maximos = df_montos_maximos.rename(columns={c: c.strip() for c in df_montos_maximos.columns})
df_montos_maximos.loc[:, 'INV3'] = (df_montos_maximos.INV3.str.strip()
                                                          .str.replace(',', '')
                                                          .astype(int))
df_montos_maximos.loc[:, 'Adjudicación directa'] = (df_montos_maximos['Adjudicación directa'].str.strip()
                                                                                             .str.replace(',', '')
                                                                                             .astype(int))
df_montos_maximos.loc[:, 'Tipo de contratación'] = (df_montos_maximos['Tipo de contratación'].str.normalize('NFD')
                                                                                             .str.encode('ascii', 'ignore')
                                                                                             .str.decode('utf-8')
                                                                                             .str.upper())
print(df_montos_maximos.shape)
df_montos_maximos.head()

porcentaje_adjudicaciones_excedieron_monto = partial(
    porcentaje_adjudicaciones_excedieron_monto, df_maximos=df_montos_maximos)
porcentaje_invitaciones_excedieron_monto = partial(
    porcentaje_invitaciones_excedieron_monto, df_maximos=df_montos_maximos)

(30, 4)


# Generación de features

In [7]:
funciones_procedimientos = {
    'general': [
        monto_por_unidad_compradora,
        proveedores_distintos,
        procedimientos_distintos,
        numero_de_contratos,
    ],
    'competencia': [
        contratos_por_proveedor,
        porcentaje_procedimientos_por_tipo,
        porcentaje_monto_tipo_procedimiento,
        importe_promedio_por_contrato,
        calcular_IHH_ID_contratos,
        calcular_IHH_ID_monto,
        tendencia_adjudicacion_directa,
        
    ],
    'transparencia': [
        porcentaje_procedimientos_presenciales,
        contratos_promedio_por_procedimimento,
        # contratos_por_duracion,
        # monto_por_duracion,
        promedio_datos_faltantes_poc_contrato,
    ],
    'anomalias': [
        interaccion_rfc_fantasma,
        interaccion_sancionados,
        porcentaje_contratos_por_convenio,
        pc_licitaciones_nacionales_menor_15_dias,
        pc_licitaciones_internacionales_menor_20_dias,
        pc_licitaciones_internacionales_menor_40_dias,
        diferente_estratificacion,
        porcentaje_adjudicaciones_excedieron_monto,
        porcentaje_invitaciones_excedieron_monto,
    ]
}


funciones_scraper = {
    'general': [
        
    ],
    'competencia': [
            
    ],
    'transparencia': [
        porcentaje_procs_sin_contrato,
        porcentaje_procs_sin_fallo,
        porcentaje_procs_sin_apertura,
        porcentaje_procedimientos_sin_archivos,
        promedio_procs_por_archivo,
        tendencia_no_publicacion_contratos,
        porcentaje_adjudicaciones_incompletas,
        porcentaje_invitaciones_incompletas,
        porcentaje_licitaciones_incompletas,
    ],
    'anomalias': [
        promedio_convenios_por_proc,
        porcentaje_procs_sin_junta_aclaracion,
        porcentaje_procs_sin_convocatoria,
    ]
    
}


funciones_participantes = {
    'general': [
        
    ],
    'competencia': [
        porcentaje_licitaciones_con_un_participante,
        procedimientos_promedio_por_participantes,
        indice_participacion,
        procedimientos_por_participantes_unicos,
        tendencia_incremento_participantes,
    ],
    'transparencia': [
        
    ],
    'anomalias': [
        
    ]
    
}


conceptos = tuple(funciones_procedimientos.keys())

tipos_contratacion = (
    'ADQUISICIONES',
    'SERVICIOS',
    'OBRA PUBLICA',
    'ARRENDAMIENTOS',
    'SERVICIOS RELACIONADOS CON LA OP'
)

for concepto in conceptos:
    for tipo in tipos_contratacion:
        # sub-tablas
        df_procs_aux = df_procedimientos.loc[
            (df_procedimientos.TIPO_CONTRATACION == tipo)
        ]
        df_scraper_aux = df_scraper.loc[
            (df_scraper.TIPO_CONTRATACION == tipo)
        ]
        df_parts_aux = df_participantes.loc[
            (df_participantes.TIPO_CONTRATACION == tipo)
        ]
        # se agarran las funciones para cada tabla
        functions_procs_in_concept = funciones_procedimientos[concepto]
        functions_scraper_in_concept = funciones_scraper[concepto]
        functions_parts_in_concept = funciones_participantes[concepto]
        print(concepto, tipo)
        # Calculo de features
        df_features_procs = [
            function(df_procs_aux, tipo_contratacion=tipo)
            for function in functions_procs_in_concept
        ]
        df_features_procs = [df.set_index('CLAVEUC')
                             for df in df_features_procs
                             if df is not None]
        df_features_scraper = [
            function(df_scraper_aux, tipo_contratacion=tipo)
            for function in functions_scraper_in_concept
        ]
        df_features_scraper = [df.set_index('CLAVEUC')
                               for df in df_features_scraper
                               if df is not None]
        df_features_parts = [
            function(df_parts_aux, tipo_contratacion=tipo)
            for function in functions_parts_in_concept
        ]
        df_features_parts = [df.set_index('CLAVEUC')
                             for df in df_features_parts
                             if df is not None]
        # print([df.shape for df in df_features_scraper])
        # join de features por tabla en concepto
        # Se asume que procedimientos es la BASE PRINCIPAL
        # Join de tablas en el concetp
        df_features = pd.concat(df_features_procs, join='inner', axis=1)
        df_features = df_features.reset_index()
        if len(df_features_scraper) > 0:
            df_features_scraper = pd.concat(df_features_scraper, join='inner', axis=1)
            df_features_scraper = df_features_scraper.reset_index()
            df_features = pd.merge(df_features, df_features_scraper, on='CLAVEUC', how='left')
        if len(df_features_parts) > 0:
            df_features_parts = pd.concat(df_features_parts, join='inner', axis=1)
            df_features_parts = df_features_parts.reset_index()
            df_features = pd.merge(df_features, df_features_parts, on='CLAVEUC', how='left')
        file_path = os.path.join('../data/conceptos/', concepto, tipo, 'features.csv')
        df_features.to_csv(file_path, index=False, quoting=1, encoding='utf-8')
        print(file_path)
        print('-' * 50)


general ADQUISICIONES
../data/conceptos/general/ADQUISICIONES/features.csv
--------------------------------------------------
general SERVICIOS
../data/conceptos/general/SERVICIOS/features.csv
--------------------------------------------------
general OBRA PUBLICA
../data/conceptos/general/OBRA PUBLICA/features.csv
--------------------------------------------------
general ARRENDAMIENTOS
../data/conceptos/general/ARRENDAMIENTOS/features.csv
--------------------------------------------------
general SERVICIOS RELACIONADOS CON LA OP
../data/conceptos/general/SERVICIOS RELACIONADOS CON LA OP/features.csv
--------------------------------------------------
competencia ADQUISICIONES
../data/conceptos/competencia/ADQUISICIONES/features.csv
--------------------------------------------------
competencia SERVICIOS
../data/conceptos/competencia/SERVICIOS/features.csv
--------------------------------------------------
competencia OBRA PUBLICA
../data/conceptos/competencia/OBRA PUBLICA/features.csv

In [9]:
[df.shape for df in df_features_parts]

[]

In [8]:
# df_features_parts[4]

In [25]:
import numpy as np
import networkx as nx
from networkx.algorithms import bipartite
from sklearn.linear_model import Ridge


df_test = tendencia_incremento_participantes(df_parts_aux)

In [ ]:
# df_algo = df_participantes.loc[df_participantes.TIPO_CONTRATACION == 'SERVICIOS']
# tendencia_incremento_participantes(df_algo)

In [24]:

df_parts_aux

,NUMERO_PROCEDIMIENTO,CODIGO_EXPEDIENTE,CLAVEUC,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,FORMA_PROCEDIMIENTO,PROVEEDOR_CONTRATISTA,ESTATUS_DE_PROPUESTA,ESTATUS_PARTIDA,ESTATUS_FALLO,PRECIO_TOTAL


In [ ]:
# df_participantes

In [9]:
df_parts_aux

,NUMERO_PROCEDIMIENTO,CODIGO_EXPEDIENTE,CLAVEUC,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,FORMA_PROCEDIMIENTO,PROVEEDOR_CONTRATISTA,ESTATUS_DE_PROPUESTA,ESTATUS_PARTIDA,ESTATUS_FALLO,PRECIO_TOTAL


In [12]:
def test(a, b, **kwargs):
    print(a, b, kwargs)
    
test('mundo', 'raul', dona='donut')

mundo raul {'dona': 'donut'}


In [15]:
prueba = partial(test, b='XYZ')
prueba('mundo', tipo='tipo')

mundo XYZ {'tipo': 'tipo'}


In [3]:
df_participantes.TIPO_CONTRATACION.value_counts(dropna=False)

ADQUISICIONES                       4044473
SERVICIOS RELACIONADOS CON LA OP     516361
SERVICIOS                            489500
OBRA PUBLICA                         138923
ARRENDAMIENTOS                        26794
SIN REPORTAR                            895
Name: TIPO_CONTRATACION, dtype: int64